In [1]:
import pandas as pd # import Pandas library
import gzip # to manage gzip-compressed files
import json # to handle json data

In [2]:
steam_games = pd.read_json('./datasets_crudos/steam_games.json.gz', compression='gzip', lines=True)
"""Reads a JSON file containing Steam games data. It specifies that the file is compressed using gzip and reads the file line by line."""

'Reads a JSON file containing Steam games data. It specifies that the file is compressed using gzip and reads the file line by line.'

In [12]:
# make a explode of column genres to keep with only one genre per item, to use later in the query where genre is taken as reference
steam_games1 = steam_games.explode(["genres"])
steam_games2 = pd.json_normalize(steam_games1['genres']).set_index(steam_games1['genres'].index)
steam_games3= pd.concat([steam_games2, steam_games1], axis = 1)

In [13]:
steam_games3.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
88310,Kotoshiro,Casual,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
88310,Kotoshiro,Indie,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
88310,Kotoshiro,Simulation,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
88310,Kotoshiro,Strategy,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro


In [19]:
steam_games3 = steam_games3.drop_duplicates(subset='id')

In [ ]:
steam_games3.head(20)

In [21]:
print(steam_games3.isna().sum()) # lot of null values this dataset

publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        1
specs            670
price           1377
early_access       0
id                 1
developer       3299
dtype: int64


In [15]:
steam_games3.dropna(how='all', inplace=True) # the how=all ensures to drop only the rows that are completely empty

In [22]:
steam_games3.isna().sum() # check na values

publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        1
specs            670
price           1377
early_access       0
id                 1
developer       3299
dtype: int64

In [23]:
steam_games3.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')

In [24]:
# drop columns which are no needed for queries
steam_games4 = steam_games3.drop(['app_name', 'url','tags','reviews_url','specs','early_access'], axis=1)

In [27]:
steam_games4.head()

,publisher,genres,title,release_date,price,item_id,developer
88310,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,4.99,761140.0,Kotoshiro
88311,"Making Fun, Inc.",Free to Play,Ironbound,2018-01-04,Free To Play,643980.0,Secret Level SRL
88312,Poolians.com,Casual,Real Pool 3D - Poolians,2017-07-24,Free to Play,670290.0,Poolians.com
88313,彼岸领域,Action,弹炸人2222,2017-12-07,0.99,767400.0,彼岸领域
88314,None,None,None,None,2.99,773570.0,None


In [26]:
# rename id to item_id to join later tables by that column
steam_games4.rename(columns={'id':'item_id'}, inplace=True)

In [32]:
# Export DataFrame to csv file to check
file_name = './dataset_ready/steam_games_ver2.csv' #file name included extension
pd.DataFrame(steam_games4, columns = steam_games4.columns).to_csv(file_name, index=False)  # "index=False" to avoid indexs columns